In [15]:

GYRO = False
SEQUENCE_LENGTH = 5
SEQUENCE_OVERLAP = 4
BATCH_SIZE = 10
EPOCHS = 50
MODEL_NAME = f"other_activity_epochs:{EPOCHS}_batch:{BATCH_SIZE}_gyro:{GYRO}_window:{SEQUENCE_LENGTH}_overlap:{SEQUENCE_OVERLAP}.keras"

# EDIT THE VALUES IN THE CELL ABOVE THEN PRESS RUN ALL

In [16]:
import helpers.file_tagger as file_tagger
import helpers.sequence_generator as sequence_generator

import tensorflow as tf
import numpy as np
from keras import layers, Sequential, models
from sklearn.model_selection import train_test_split


In [17]:
DATA_DIRECTORY = "./all_respeck"
OTHER_ACTIVITIES = {
    "sitting&talking",
    "sitting&eating",
    "sitting&singing",
    "sitting&laughing",
    
    "standing&talking",
    "standing&eating",
    "standing&singing",
    "standing&laughing",
    
    "lying_down_back&talking",
    "lying_down_back&singing",
    "lying_down_back&laughing",
    
    "lying_down_right&talking",
    "lying_down_right&singing",
    "lying_down_right&laughing",
    
    "lying_down_left&talking",
    "lying_down_left&singing",
    "lying_down_left&laughing",
    
    "lying_down_stomach&talking",
    "lying_down_stomach&singing",
    "lying_down_stomach&laughing",
}

In [18]:
def generate_training_data(directory, sequence_length, overlap, gyro = GYRO): # if gyro is false, only accelerometer data is used

    tagged_data = []

    # group each csv file into their respective areas
    csv_dictionary = file_tagger.tag_directory(directory)

    # iterates through each activity
    for key in OTHER_ACTIVITIES:

        # iterates through each csv file for the activity 
        for csv_file in csv_dictionary[key]:
            if gyro:
                sequences = sequence_generator.generate_sequences_from_file_with_gyroscope(directory + "/" + csv_file, sequence_length, overlap)
            else:
                sequences = sequence_generator.generate_sequences_from_file_without_gyroscope(directory + "/" + csv_file, sequence_length, overlap)

            # iterate through each generated sequence
            for sequence in sequences:
                tagged_data.append((key, sequence))

    print ("there are " + str(len(tagged_data)) + " tagged sequences in the dataset")
    return tagged_data

In [19]:
# split data into training, dev, and test sets
def train_dev_test_split(data, labels, dev_size, test_size, random_state=42):
    # Split the data into training and temporary (dev + test) sets
    train_data, temp_data, train_labels, temp_labels = train_test_split(data, labels, test_size=(dev_size + test_size), random_state=random_state)
    
    # Split the temporary data into dev and test sets
    dev_data, test_data, dev_labels, test_labels = train_test_split(temp_data, temp_labels, 
                                                                 test_size=(test_size / (dev_size + test_size)), random_state=random_state)
    
    return train_data, dev_data, test_data, train_labels, dev_labels, test_labels

In [20]:
tagged_sequences = generate_training_data(DATA_DIRECTORY, SEQUENCE_LENGTH, SEQUENCE_OVERLAP)

# Combine all sequences and labels
sequences = [sequence for _, sequence in tagged_sequences]
labels = [label for label, _ in tagged_sequences]


# encode labels to numbers
sequences = np.array(sequences, dtype=np.float32)
label_to_index = {label: idx for idx, label in enumerate(OTHER_ACTIVITIES)}
labels_encoded = [label_to_index[label] for label in labels]
labels_encoded = np.array(labels_encoded)

there are 23873 tagged sequences in the dataset


In [21]:
#CNN MODEL
def train_model_CNN(input_data, labels_encoded, unique_labels, epochs, batch_size, validation_data):
    if GYRO:
        width = 6
    else:
        width = 3
    # Define the CNN model for your specific input shape
    model = Sequential([
        layers.Conv1D(32, 3, activation='relu', input_shape=(SEQUENCE_LENGTH*25, width)),
        layers.MaxPooling1D(2),
        layers.Conv1D(64, 3, activation='relu'),
        layers.MaxPooling1D(2),
        layers.Conv1D(64, 3, activation='relu'),
        layers.MaxPooling1D(2),
        layers.Conv1D(128, 3, activation='relu'),
        layers.MaxPooling1D(2),
        layers.Dropout(0.5),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(len(unique_labels), activation='softmax')
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the CNN model
    model.fit(input_data, labels_encoded, epochs=epochs, batch_size=batch_size, validation_data=validation_data)

    return model

In [22]:

train_data, dev_data, test_data, train_labels, dev_labels, test_labels = train_dev_test_split(sequences, labels_encoded, dev_size=0.1, test_size=0.1) #10% dev, 10% test


# train and save model (CHOOSE BETWEEN CNN AND LSTM)
model = train_model_CNN(train_data, train_labels, OTHER_ACTIVITIES, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(dev_data, dev_labels)) #batch_size, epochs


# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_data, test_labels)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

# Save the trained model
model.save(f"models/Task3/{MODEL_NAME}_{test_accuracy}")

Epoch 1/50
1910/1910 [==============================] - 9s 4ms/step - loss: 1.6625 - accuracy: 0.2561 - val_loss: 1.5268 - val_accuracy: 0.2878
Epoch 2/50
1910/1910 [==============================] - 8s 4ms/step - loss: 1.5148 - accuracy: 0.2963 - val_loss: 1.3371 - val_accuracy: 0.3758
Epoch 3/50
1910/1910 [==============================] - 8s 4ms/step - loss: 1.2845 - accuracy: 0.3993 - val_loss: 1.0842 - val_accuracy: 0.5057
Epoch 4/50
1910/1910 [==============================] - 8s 4ms/step - loss: 1.1132 - accuracy: 0.4771 - val_loss: 1.0089 - val_accuracy: 0.5228
Epoch 5/50
1910/1910 [==============================] - 7s 4ms/step - loss: 1.0703 - accuracy: 0.5018 - val_loss: 1.1149 - val_accuracy: 0.4897
Epoch 6/50
1910/1910 [==============================] - 8s 4ms/step - loss: 1.0366 - accuracy: 0.5182 - val_loss: 1.0313 - val_accuracy: 0.5220
Epoch 7/50
1910/1910 [==============================] - 8s 4ms/step - loss: 1.0271 - accuracy: 0.5220 - val_loss: 1.0090 - val_accuracy:

2023-11-03 19:57:32.740566: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,5,128]
	 [[{{node inputs}}]]
2023-11-03 19:57:33.135207: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,5,128]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: models/Task3/other_activity_epochs:50_batch:10_gyro:False_window:5_overlap:4.keras_0.6930485963821411/assets


INFO:tensorflow:Assets written to: models/Task3/other_activity_epochs:50_batch:10_gyro:False_window:5_overlap:4.keras_0.6930485963821411/assets
